##### Imports, setup

In [2]:
import rasterio
from rasterio.plot import reshape_as_image
import requests
import tensorflow as tf

file_id = '1o76QoBtn6ExxO8KgcCqdOiun_KsWoMJl'
in_path = 'in.jp2'
out_path = 'out.tiff'

ModuleNotFoundError: No module named 'matplotlib'

##### Get satellite image

In [30]:
# From Part_1_GettingStarted.ipynb
# Which is in turn from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def get_file(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value
        
        return None
    
    def save_response_content(response, destination):
        CHUNK_SIZE = 32768
        
        with open(destination, 'wb') as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # Filter out keep-alive new chunks
                    f.write(chunk)
    
    URL = 'https://docs.google.com/uc?export=download'
    
    session = requests.Session()
    
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    
    save_response_content(response, destination)

get_file(file_id, in_path)

##### Decode

In [1]:
def read_image(input):
    with rasterio.open(input) as dataset:
        return dataset.read()

raster = read_image(in_path)
image = reshape_as_image(raster)
image.shape

NameError: name 'in_path' is not defined

##### Do some image manipulation with TF/Keras

In [32]:
def manipulate_image(image):
    # TODO: how to make numpy array from Rasterio into something TF can work on?
    tensor_image = tf.convert_to_tensor(image)
    
    # Scale down
    scale_width = int(image.shape[0] / 4)
    scale_height = int(image.shape[1] / 4)
    scale_size = [scale_width, scale_height]
    tensor_image_resize = tf.image.resize(tensor_image, scale_size, 'nearest')
    
    # Rotate
    tensor_image_rot = tf.image.rot90(tensor_image_resize)
    
    return tensor_image_rot
    
new_image = manipulate_image(image)
new_image.shape

TensorShape([2745, 2745, 3])

##### Save changed image

In [38]:
def save_image(output, image):
    with rasterio.open(
        out_path,
        'w',
        driver='GTiff',
        height=image.shape[1],
        width=image.shape[2],
        count=image.shape[0],
        dtype=image.dtype) as f:
        f.write(image)

raster2 = reshape_as_raster(new_image)
save_image(out_path, raster2)